# Walmart 구매 데이터 분석 보고서 (가설 검정 포함)

목표는 간단하다.  
이 데이터로 가설을 세우고, 그래프와 통계 검정으로 확인하고, 마지막에 말이 되는 결론을 남긴다.

- 데이터: 사용자/도시/상품 카테고리/구매금액
- 주의: City_Category, Occupation, Product_Category는 실제 의미가 숨겨진(마스킹) 값이라 해석을 과하게 하면 위험하다.

## 0. 환경 설정

한글이 그래프에서 깨지는 문제부터 먼저 잡는다.  
그리고 이후 셀들이 좀 깔끔하게 보이도록 기본 설정을 해둔다.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

# 한글 폰트 설정 (환경마다 다르니 가능한 폰트 후보를 순서대로 시도)
from matplotlib import font_manager, rcParams

def set_korean_font():
    candidates = ["Malgun Gothic", "AppleGothic", "NanumGothic", "Noto Sans CJK KR", "Noto Sans KR"]
    available = {f.name for f in font_manager.fontManager.ttflist}
    for c in candidates:
        if c in available:
            rcParams["font.family"] = c
            break
    rcParams["axes.unicode_minus"] = False

set_korean_font()

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (9, 5)

## 1. 데이터 불러오기

여기서 파일 경로만 맞으면 바로 시작 가능하다.  
(내 로컬에서 파일명이 다르면 여기만 바꾸면 됨)

In [ ]:
# 파일 경로
# df = pd.read_csv("walmart.csv")
df = pd.read_csv("walmart.csv")

df.head()

## 2. 데이터 점검

- 결측치 확인
- 컬럼 타입 확인
- 요약 통계로 Purchase 스케일 확인

이 단계는 지루한데, 여기서 실수 줄이면 뒤가 편하다.

In [ ]:
df.info()

In [ ]:
# 결측치
df.isnull().sum()

In [ ]:
# 기본 요약
df.describe(include="all").T

## 3. 전처리 (분석에 필요한 최소한)

핵심은 2가지다.

1) 숫자인 척하는 범주형을 category로 바꾸기  
2) 순서가 있는 범주(Age, Stay_In_Current_City_Years)는 ordered category로 바꾸기

In [ ]:
# 컬럼명 공백 방지
df.columns = df.columns.str.strip()

# 숫자인 척하는 범주형
for col in ["Occupation", "Product_Category", "Marital_Status"]:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Age는 순서형 범주
age_order = ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
if "Age" in df.columns:
    df["Age"] = pd.Categorical(df["Age"], categories=age_order, ordered=True)

# Stay_In_Current_City_Years도 순서형 범주
stay_order = ['0', '1', '2', '3', '4+']
if "Stay_In_Current_City_Years" in df.columns:
    df["Stay_In_Current_City_Years"] = pd.Categorical(
        df["Stay_In_Current_City_Years"], categories=stay_order, ordered=True
    )

df.dtypes

## 4. 빠른 개요 (Executive Summary용 숫자)

보고서 앞부분에 넣을 만한 숫자만 뽑는다.

In [ ]:
summary = pd.DataFrame({
    "총 거래 수": [len(df)],
    "총 고객 수": [df["User_ID"].nunique()],
    "총 상품 수": [df["Product_ID"].nunique()],
    "총 매출(합계 Purchase)": [df["Purchase"].sum()],
    "평균 구매금액": [df["Purchase"].mean()],
    "중앙값 구매금액": [df["Purchase"].median()],
})
summary

# 가설 검정 파트

규칙 하나만 지키자.

- 가설은 검증 가능한 문장으로 쓴다
- 해석은 단정하지 말고, 데이터가 말하는 만큼만 말한다

## 공통 함수 (효과크기 포함)

p-value만 보면 너무 차갑다.  
그래서 효과크기도 같이 본다. (작아도 유의할 수 있고, 커도 표본이 작으면 유의가 안 뜰 수 있으니)

In [ ]:
def epsilon_squared_kw(H, n, k):
    # Kruskal-Wallis effect size (epsilon squared)
    # 참고: H는 kw 통계량, n 전체 샘플, k 그룹 수
    return (H - k + 1) / (n - k)

def cramers_v(confusion_matrix):
    chi2 = stats.chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.values.sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r-1, k-1))))

def bonferroni(p, m):
    return min(p * m, 1.0)

## 가설 1. City_Category 유형에 따라 구매금액 분포는 차이가 있는가?

가설 설정
- H0: City_Category(A/B/C) 간 구매금액(Purchase) 분포는 동일하다.
- H1: City_Category 유형에 따라 구매금액 분포는 동일하지 않다.

여기서 중요한 포인트:
- City_Category가 '대도시/중소도시' 같은 뜻이라고 단정하지 않는다.
- 그냥 마스킹된 유형 A/B/C라고만 두고, 분포 차이 여부만 본다.

In [ ]:
# 시각화: 분포 비교
sns.boxplot(data=df, x="City_Category", y="Purchase")
plt.title("City_Category별 구매금액 분포 (boxplot)")
plt.xlabel("City_Category")
plt.ylabel("Purchase")
plt.show()

In [ ]:
# 요약 통계: mean/median/std 같이 보기
city_stats = df.groupby("City_Category")["Purchase"].agg(
    거래수="count",
    평균="mean",
    중앙값="median",
    표준편차="std"
).sort_index()
city_stats

In [ ]:
# Kruskal-Wallis (비모수): 분포 차이 검정
groups = [g["Purchase"].values for _, g in df.groupby("City_Category")]
kw = stats.kruskal(*groups)

n = len(df)
k = df["City_Category"].nunique()
eps2 = epsilon_squared_kw(kw.statistic, n, k)

pd.DataFrame({
    "검정": ["Kruskal-Wallis"],
    "통계량(H)": [kw.statistic],
    "p-value": [kw.pvalue],
    "효과크기(epsilon^2)": [eps2]
})

가설 1 보조 분석: 어떤 쌍이 다른지(사후 비교)

Kruskal-Wallis가 유의하면, 최소 한 쌍은 다르다는 뜻이다.  
그래서 A-B, A-C, B-C를 Mann-Whitney U로 비교하고, Bonferroni로 보수적으로 보정한다.

In [ ]:
# Pairwise Mann-Whitney U + Bonferroni
cats = list(df["City_Category"].dropna().unique())
cats.sort()

pairs = []
m = len(cats) * (len(cats) - 1) // 2

for i in range(len(cats)):
    for j in range(i+1, len(cats)):
        c1, c2 = cats[i], cats[j]
        x = df.loc[df["City_Category"] == c1, "Purchase"]
        y = df.loc[df["City_Category"] == c2, "Purchase"]
        u = stats.mannwhitneyu(x, y, alternative="two-sided")
        pairs.append({
            "비교": f"{c1} vs {c2}",
            "U": u.statistic,
            "p_raw": u.pvalue,
            "p_bonf": bonferroni(u.pvalue, m)
        })

pairwise_city = pd.DataFrame(pairs).sort_values("p_bonf")
pairwise_city

가설 1 해석 메모 (보고서 톤)

- 중앙값이 더 큰 그룹이 있다고 해서, 그 그룹이 '더 큰 도시'라고 말할 수는 없다.
- 대신, City_Category별로 구매금액의 전형적인 수준(중앙값)과 분산(표준편차)이 다를 수 있다고 정리한다.
- 유의성(p-value)과 효과크기(epsilon^2)를 같이 보고, 차이가 "있다/없다"를 과하게 단정하지 않는다.

## 가설 2. 도시 거주 기간에 따라 구매금액 특성은 달라지는가?

가설 설정
- H0: 거주 기간(Stay_In_Current_City_Years)별 구매금액 분포는 동일하다.
- H1: 거주 기간별 구매금액 분포는 동일하지 않다.

내가 여기서 하고 싶은 건 인과가 아니다.  
"오래 살면 더 쓴다" 같은 말을 하고 싶은 게 아니라,  
거주 기간 그룹별로 분포가 실제로 달라 보이는지만 확인한다.

In [ ]:
# 시각화
sns.boxplot(data=df, x="Stay_In_Current_City_Years", y="Purchase")
plt.title("거주 기간별 구매금액 분포 (boxplot)")
plt.xlabel("Stay_In_Current_City_Years")
plt.ylabel("Purchase")
plt.show()

In [ ]:
stay_stats = df.groupby("Stay_In_Current_City_Years")["Purchase"].agg(
    거래수="count",
    평균="mean",
    중앙값="median",
    표준편차="std"
)
stay_stats

In [ ]:
# Kruskal-Wallis
stay_groups = [g["Purchase"].values for _, g in df.groupby("Stay_In_Current_City_Years")]
kw2 = stats.kruskal(*stay_groups)

n2 = len(df)
k2 = df["Stay_In_Current_City_Years"].nunique()
eps2_2 = epsilon_squared_kw(kw2.statistic, n2, k2)

pd.DataFrame({
    "검정": ["Kruskal-Wallis"],
    "통계량(H)": [kw2.statistic],
    "p-value": [kw2.pvalue],
    "효과크기(epsilon^2)": [eps2_2]
})

가설 2 보조 검정: 단조 경향이 있는지 (Spearman)

거주 기간은 순서형이니까, 단조 증가/감소 경향이 있는지 Spearman 상관으로 본다.  
(값 매핑은 0,1,2,3,4로 두고 4+는 4로 본다)

In [ ]:
# 순서형을 숫자로 매핑
stay_map = {"0":0, "1":1, "2":2, "3":3, "4+":4}
stay_numeric = df["Stay_In_Current_City_Years"].astype(str).map(stay_map)

rho, p = stats.spearmanr(stay_numeric, df["Purchase"])
pd.DataFrame({
    "검정": ["Spearman rank correlation"],
    "rho": [rho],
    "p-value": [p]
})

가설 2 보조 검정: 분산 차이 (Levene)

"거주 기간이 길수록 분산이 커진다" 같은 말을 하고 싶다면,
최소한 분산이 그룹별로 다르다는 근거부터 확인해야 한다.

In [ ]:
# Levene test for equal variances (중앙값 기반으로 robust)
levene = stats.levene(*stay_groups, center="median")
pd.DataFrame({
    "검정": ["Levene (median center)"],
    "통계량": [levene.statistic],
    "p-value": [levene.pvalue]
})

가설 2 해석 메모 (보고서 톤)

- Kruskal로 분포 차이를 먼저 보고,
- Spearman으로 순서형 경향을 보며,
- Levene으로 분산 차이를 확인한다.

이렇게 3개를 같이 보면, "있어 보이는 해석"이 아니라 "근거 있는 해석"에 가까워진다.

## 가설 3. 제품 카테고리별로 소비가 집중되는 고객 특성이 존재하는가?

가설 설정 (2개로 나눠서 본다)
- 3-A: Product_Category와 Gender는 독립인가?
  - H0: 독립이다
  - H1: 독립이 아니다

- 3-B: Product_Category와 Age는 독립인가?
  - H0: 독립이다
  - H1: 독립이 아니다

여기서는 Purchase 금액 자체보다,
어떤 고객 특성이 어떤 카테고리에서 더 자주 나타나는지(분포 편중)를 본다.

In [ ]:
# 3-A: Product_Category x Gender
ct_gender = pd.crosstab(df["Product_Category"], df["Gender"])
ct_gender.head()

In [ ]:
chi2, p, dof, expected = stats.chi2_contingency(ct_gender)
v = cramers_v(ct_gender)

pd.DataFrame({
    "검정": ["Chi-square (Product_Category x Gender)"],
    "chi2": [chi2],
    "dof": [dof],
    "p-value": [p],
    "효과크기(Cramer's V)": [v]
})

In [ ]:
# 시각화: 카테고리별 성별 비율
gender_ratio = ct_gender.div(ct_gender.sum(axis=1), axis=0).reset_index()
gender_ratio_melt = gender_ratio.melt(id_vars="Product_Category", var_name="Gender", value_name="비율")

sns.barplot(data=gender_ratio_melt, x="Product_Category", y="비율", hue="Gender")
plt.title("Product_Category별 Gender 비율")
plt.xlabel("Product_Category")
plt.ylabel("비율")
plt.legend(title="Gender")
plt.show()

In [ ]:
# 3-B: Product_Category x Age
ct_age = pd.crosstab(df["Product_Category"], df["Age"])
ct_age.iloc[:5, :5]

In [ ]:
chi2_a, p_a, dof_a, expected_a = stats.chi2_contingency(ct_age)
v_a = cramers_v(ct_age)

pd.DataFrame({
    "검정": ["Chi-square (Product_Category x Age)"],
    "chi2": [chi2_a],
    "dof": [dof_a],
    "p-value": [p_a],
    "효과크기(Cramer's V)": [v_a]
})

In [ ]:
# 시각화: 카테고리별 연령 분포(비율)
age_ratio = ct_age.div(ct_age.sum(axis=1), axis=0).reset_index()
age_ratio_melt = age_ratio.melt(id_vars="Product_Category", var_name="Age", value_name="비율")

# 너무 빽빽하면 상위 카테고리 몇 개만 보고 싶을 수 있으니, 기본은 전체로 두되 figsize를 키움
plt.figure(figsize=(12, 5))
sns.lineplot(data=age_ratio_melt, x="Age", y="비율", hue="Product_Category", marker="o", legend=False)
plt.title("Product_Category별 Age 분포(비율) - 전체 라인(범례 생략)")
plt.xlabel("Age")
plt.ylabel("비율")
plt.show()

가설 3 해석 메모 (보고서 톤)

- Chi-square가 유의하면, "독립이 아니다"까지는 말할 수 있다.
- 그런데 그게 곧 "차이가 크다"는 뜻은 아니다. 그래서 Cramer's V를 같이 본다.
- V가 작으면, 통계적으로는 다르지만 실무적으로는 약한 차이일 수 있다.

# 결론 및 시사점 (조금 더 제대로)

여기서는 한 문장으로 끝내지 말고,
가설 1~3을 연결해서 '무엇을 배웠는지'를 남긴다.

## 결론 요약

1) City_Category별로 구매금액 분포가 달라질 수 있다  
- 중앙값이 높은 유형이 있는 반면, 분산이 큰 유형도 있다.
- 이 차이는 "도시 규모" 같은 의미 해석으로 점프하지 않고, '유형 간 소비 구조 차이'로만 정리하는 게 안전하다.

2) 거주 기간은 단독 변수로는 설명력이 제한될 수 있다  
- 분포 차이, 경향, 분산을 같이 봤을 때 일관된 방향성이 약하면,
  거주 기간만으로 소비를 설명하기는 어렵다.
- 대신 City_Category나 Product_Category 같은 변수와 결합해서 보는 쪽이 더 낫다.

3) 제품 카테고리에는 고객 특성의 편중이 존재할 수 있다  
- Product_Category와 Gender/Age의 독립성 검정이 유의하면,
  카테고리마다 주 소비자 집단이 갈릴 가능성이 있다.
- 다만 효과크기(Cramer's V)가 작으면, "엄청난 차이"로 말하지 말고 "약한 편중" 정도로 정리하는 편이 맞다.

## 실무적으로 바꿔볼 수 있는 액션 아이디어

- City_Category별로 프로모션 설계를 다르게 해볼 수 있다
  - 중앙값이 높은 유형: 기본 단가 높은 번들/세트 전략
  - 분산이 큰 유형: 상단(고액) 고객 대상 업셀/프리미엄 프로모션

- 거주 기간은 단독 타겟팅 기준으로 쓰기 전에 재검토
  - 거주 기간만 보지 말고, City_Category x Product_Category 같이 묶어서 세그먼트를 잡는 게 더 설득력 있다

- Product_Category별 핵심 고객군이 보이면, 메시지/채널을 미세조정
  - 예: 특정 카테고리는 특정 연령대 비중이 높다 → 그 연령대가 많이 쓰는 채널과 카피 톤을 맞추기

## 오늘 회고 느낌

- 분석에서 제일 위험한 건 '내가 알고 있다고 착각하는 해석'이었다.
- 마스킹 변수는 특히 더. A가 대도시다 같은 말을 하고 싶어도, 데이터가 그 말을 해주진 않는다.
- 대신, 내가 할 수 있는 건 "차이가 있는지"를 먼저 확인하고, 효과크기까지 보고, 말할 수 있는 만큼만 말하는 것.

다음 단계로는
- City_Category x Product_Category로 2차 세그먼트 만들고,
- 각 세그먼트의 구매금액 분포와 상위 매출 기여도를 보는 걸 해보고 싶다.